In [199]:
#Import for model and responses
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser

#Import for PDF text extraction and splitting
from langchain_community.document_loaders import PyMuPDFLoader
from pypdf import PdfReader
from langchain_text_splitters import RecursiveCharacterTextSplitter

#Import for vector database and embeddings
import os
from langchain_community.embeddings.ollama import OllamaEmbeddings
from langchain_chroma import Chroma
from uuid import uuid4
from langchain_core.documents import Document

#Import for keyword extraction
import yake

#Import for relative current working directory
import os
import sys

#Import for BERTScore calculation
from transformers import BertTokenizer, BertModel
from bert_score import BERTScorer

#Imports for dataset calculation
import pandas as pd


In [200]:
#Initialising LLM 
llm = ChatOllama(model="llama3")

#Initialising the Embedding model
embedding_model = OllamaEmbeddings(model='nomic-embed-text')

#Initialising the keyword extraction model
kw_extractor = yake.KeywordExtractor()
language = "en"

#Initialising Current Working Directory
app_dir = r"C:\Users\KD\Desktop\Git\hsc-llm"

In [201]:
#Function that loads or creates a vector store
def create_or_load_vector_store(embeddings, store_name):
    persistent_directory = os.path.join(app_dir, store_name)
    if not os.path.exists(persistent_directory):
        print("creating vector store")
        vector_store = Chroma.from_documents(embeddings, persist_directory=persistent_directory)
    else:
        print("loading existing vector store")
        vector_store = Chroma(
        persist_directory=persistent_directory,
        embedding_function=embeddings
        )
    return vector_store

vector_store = create_or_load_vector_store(embedding_model, "chroma_db")

loading existing vector store


In [202]:
def get_response(query, context):
    qa_template = """
    You are an assistant for question-answering tasks.

    Use the following documents that is retrieved from the database is relevant \
    use it to provide a complete and concise response to the user's query. \
    Do not mention references, sources, or citations in your response

    If the documents provided are not relevant to the question, use your own knowledge to answer.

    Limit your answer to 3-4 sentences.

    User question: {user_questions}

    Documents: {documents}

    Answer:
    """

    qa_prompt = ChatPromptTemplate.from_template(qa_template)

    llm = ChatOllama(model="llama3")

    chain = qa_prompt | llm | StrOutputParser()

    return chain.invoke({
        "user_questions" : query,
        "documents" : context
    })


In [203]:
def rewrite_query(query: str):

    query_rewriting_str = """
    You are an expert at reformulating questions. \
    Your reformulated questions are understandable for high students and teachers.
    The question you reformulate will begin and end with with ’**’. 
    
    Question: 
    {question} 

    Only reply using a complete sentence and only give the answer in the following format:
    **Question**"""

    query_rewriting_prompt = ChatPromptTemplate.from_template(query_rewriting_str)

    chain = query_rewriting_prompt | llm | StrOutputParser()

    response = chain.invoke({
        "question" : query
    })

    print(response)

    return response 

In [204]:
def get_keywords(rewritten_query):
    keywords = kw_extractor.extract_keywords(rewritten_query)
    for kw in keywords:
        print(kw)
    return keywords

In [205]:
def perform_retrieval(vector_store, query, k=5):
    retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": k})
    results = retriever.invoke(query)
    return results

In [206]:
#Imports for dataset calculation
import pandas as pd

goldenset = pd.read_csv(r"C:\Users\KD\Desktop\Git\hsc-llm\ground_truth.csv", index_col=0)

In [207]:
goldenset.head()

,QUESTION,ANSWER
ID,,
1,Define 'asexual reproduction',Asexual reproduction is a type of reproductive...
2,Identify the sort of cell division that is inv...,The cell division in asexual reproduction is m...
3,Outline the ideal environmental conditions for...,Stable and uniform environments with a good su...
4,List the key events in binary fission for bact...,The key events that happen in binary fission f...
5,What is the difference between a somatic cell ...,Somatic cells are all the diploid cells in the...


In [208]:
questions = goldenset['QUESTION'].values.tolist()
reference_answers = goldenset['ANSWER'].values.tolist()

### Generating AI responses (NO AI QUERY REWRITING) MODEL 1

In [209]:
generated_answers = [] 
for question in questions:
    response = get_response(question, perform_retrieval(vector_store, question))
    generated_answers.append(response)

In [210]:
len(generated_answers)

162

#### Computing BERTSCORE FOR MODEL 1

In [211]:
scorer = BERTScorer(model_type='bert-base-uncased')

In [212]:
precision = []
recall = []
f1 = []
for x in range(len(generated_answers)):
    P, R, F1 = scorer.score([generated_answers[x]], [reference_answers[x]])
    precision.append(P.item())
    recall.append(R.item())   
    f1.append(F1.item())

In [213]:
precision = [round(p, 3) for p in precision]
recall = [round(r, 3) for r in recall]
f1 = [round(f, 3) for f in f1]

In [214]:
for i in range(len(precision)):
    print(f"Question {i+1} - Precision: {precision[i]} Recall: {recall[i]} F1: {f1[i]}")

Question 1 - Precision: 0.726 Recall: 0.725 F1: 0.725
Question 2 - Precision: 0.569 Recall: 0.71 F1: 0.631
Question 3 - Precision: 0.581 Recall: 0.618 F1: 0.599
Question 4 - Precision: 0.659 Recall: 0.666 F1: 0.663
Question 5 - Precision: 0.631 Recall: 0.687 F1: 0.658
Question 6 - Precision: 0.594 Recall: 0.555 F1: 0.574
Question 7 - Precision: 0.676 Recall: 0.615 F1: 0.644
Question 8 - Precision: 0.528 Recall: 0.549 F1: 0.538
Question 9 - Precision: 0.674 Recall: 0.618 F1: 0.645
Question 10 - Precision: 0.681 Recall: 0.644 F1: 0.662
Question 11 - Precision: 0.61 Recall: 0.633 F1: 0.621
Question 12 - Precision: 0.596 Recall: 0.593 F1: 0.595
Question 13 - Precision: 0.605 Recall: 0.603 F1: 0.604
Question 14 - Precision: 0.546 Recall: 0.642 F1: 0.59
Question 15 - Precision: 0.528 Recall: 0.672 F1: 0.591
Question 16 - Precision: 0.543 Recall: 0.553 F1: 0.548
Question 17 - Precision: 0.461 Recall: 0.639 F1: 0.536
Question 18 - Precision: 0.556 Recall: 0.563 F1: 0.56
Question 19 - Precision

In [215]:
goldenset = pd.read_csv(r"C:\Users\KD\Desktop\Git\hsc-llm\ground_truth.csv", index_col=0)

In [216]:
goldenset

,QUESTION,ANSWER
ID,,
1,Define 'asexual reproduction',Asexual reproduction is a type of reproductive...
2,Identify the sort of cell division that is inv...,The cell division in asexual reproduction is m...
3,Outline the ideal environmental conditions for...,Stable and uniform environments with a good su...
4,List the key events in binary fission for bact...,The key events that happen in binary fission f...
5,What is the difference between a somatic cell ...,Somatic cells are all the diploid cells in the...
...,...,...
158,Explain how the eye detects different colours.,"There are three types of cone cells, each with..."
159,Discuss the value of having binocular vision.,Binocular vision requires two eyes quite close...
160,Arrange the following structures of the excret...,"The largest is kidney, then nephron, glomerulu..."


In [217]:
index_list = []
for i in range(1, len(goldenset)+1):
    index_list.append(i)

In [218]:
len(index_list)

162

In [219]:
generated_col = pd.DataFrame({'ID':index_list,'Generated Answer': generated_answers, 'Precision': precision, 'Recall': recall, 'F1': f1})

In [220]:
generated_col.set_index('ID', inplace=True)

In [221]:
generated_col

,Generated Answer,Precision,Recall,F1
ID,,,,
1,Asexual reproduction is the production of offs...,0.726,0.725,0.725
2,The sort of cell division involved in asexual ...,0.569,0.710,0.631
3,The ideal environmental conditions for asexual...,0.581,0.618,0.599
4,The key events in binary fission for bacteria ...,0.659,0.666,0.663
5,A somatic cell is a non-reproductive cell foun...,0.631,0.687,0.658
...,...,...,...,...
158,The eye detects different colors through the c...,0.648,0.668,0.658
159,Having binocular vision is essential for human...,0.680,0.583,0.628
160,Here is a complete and concise response to the...,0.575,0.751,0.651


#### Creating Pandas Dataframe to store calculations in CSV

In [222]:
goldenset = pd.concat([goldenset, generated_col], axis=1)

In [223]:
goldenset

,QUESTION,ANSWER,Generated Answer,Precision,Recall,F1
ID,,,,,,
1,Define 'asexual reproduction',Asexual reproduction is a type of reproductive...,Asexual reproduction is the production of offs...,0.726,0.725,0.725
2,Identify the sort of cell division that is inv...,The cell division in asexual reproduction is m...,The sort of cell division involved in asexual ...,0.569,0.710,0.631
3,Outline the ideal environmental conditions for...,Stable and uniform environments with a good su...,The ideal environmental conditions for asexual...,0.581,0.618,0.599
4,List the key events in binary fission for bact...,The key events that happen in binary fission f...,The key events in binary fission for bacteria ...,0.659,0.666,0.663
5,What is the difference between a somatic cell ...,Somatic cells are all the diploid cells in the...,A somatic cell is a non-reproductive cell foun...,0.631,0.687,0.658
...,...,...,...,...,...,...
158,Explain how the eye detects different colours.,"There are three types of cone cells, each with...",The eye detects different colors through the c...,0.648,0.668,0.658
159,Discuss the value of having binocular vision.,Binocular vision requires two eyes quite close...,Having binocular vision is essential for human...,0.680,0.583,0.628
160,Arrange the following structures of the excret...,"The largest is kidney, then nephron, glomerulu...",Here is a complete and concise response to the...,0.575,0.751,0.651


In [224]:
#exporting the dataframe to a csv file
goldenset.to_csv(r'C:\Users\KD\Desktop\Git\hsc-llm\berteval-model1.csv')

### Generating AI responses (with AI QUERY REWRITING) MODEL 2

In [225]:
generated_answers = [] 
for question in questions:
    reworded_query = rewrite_query(question)        
    test_response = get_response(reworded_query, perform_retrieval(vector_store, reworded_query))
    generated_answers.append(test_response)

**What is meant by asexual reproduction, where an organism produces offspring that are genetically identical to itself without the involvement of another individual, such as budding or spore formation?**
**What type of cell division process, characterized by the replication of DNA followed by the separation of daughter cells, occurs in organisms such as bacteria, yeast, and some protists to produce genetically identical offspring through asexual reproduction?**
**What are the optimal environmental circumstances that facilitate asexual reproduction in various organisms, and what factors make these conditions conducive for successful asexual propagation?**
**What are the crucial stages that occur during bacterial binary fission, including the replication of DNA, separation of chromosomes, and formation of two identical daughter cells?**
**What is the fundamental distinction between a somatic cell, which makes up most of our body tissue, and a gamete, such as a sperm or egg cell, that pla

In [226]:
len(generated_answers)

162

#### Computing BERTSCORE FOR MODEL 2

In [227]:
scorer = BERTScorer(model_type='bert-base-uncased')

In [228]:
precision = []
recall = []
f1 = []
for x in range(len(generated_answers)):
    P, R, F1 = scorer.score([generated_answers[x]], [reference_answers[x]])
    precision.append(P.item())
    recall.append(R.item())   
    f1.append(F1.item())

In [229]:
precision = [round(p, 3) for p in precision]
recall = [round(r, 3) for r in recall]
f1 = [round(f, 3) for f in f1]

In [230]:
for i in range(len(precision)):
    print(f"Question {i+1} - Precision: {precision[i]} Recall: {recall[i]} F1: {f1[i]}")

Question 1 - Precision: 0.652 Recall: 0.765 F1: 0.704
Question 2 - Precision: 0.555 Recall: 0.673 F1: 0.608
Question 3 - Precision: 0.632 Recall: 0.62 F1: 0.626
Question 4 - Precision: 0.628 Recall: 0.636 F1: 0.632
Question 5 - Precision: 0.592 Recall: 0.665 F1: 0.626
Question 6 - Precision: 0.571 Recall: 0.656 F1: 0.61
Question 7 - Precision: 0.686 Recall: 0.616 F1: 0.649
Question 8 - Precision: 0.551 Recall: 0.579 F1: 0.565
Question 9 - Precision: 0.707 Recall: 0.735 F1: 0.72
Question 10 - Precision: 0.646 Recall: 0.684 F1: 0.665
Question 11 - Precision: 0.548 Recall: 0.621 F1: 0.582
Question 12 - Precision: 0.63 Recall: 0.607 F1: 0.618
Question 13 - Precision: 0.666 Recall: 0.643 F1: 0.654
Question 14 - Precision: 0.544 Recall: 0.603 F1: 0.572
Question 15 - Precision: 0.522 Recall: 0.635 F1: 0.573
Question 16 - Precision: 0.528 Recall: 0.569 F1: 0.548
Question 17 - Precision: 0.477 Recall: 0.656 F1: 0.552
Question 18 - Precision: 0.52 Recall: 0.566 F1: 0.542
Question 19 - Precision:

In [231]:
index_list = []
for i in range(1, len(goldenset)+1):
    index_list.append(i)

In [232]:
len(index_list)

162

In [233]:
goldenset = pd.read_csv(r"C:\Users\KD\Desktop\Git\hsc-llm\ground_truth.csv", index_col=0)

In [234]:
generated_col = pd.DataFrame({'ID':index_list,'Generated Answer': generated_answers, 'Precision': precision, 'Recall': recall, 'F1': f1})

In [235]:
generated_col.set_index('ID', inplace=True)

In [236]:
generated_col

,Generated Answer,Precision,Recall,F1
ID,,,,
1,Asexual reproduction is a process where an org...,0.652,0.765,0.704
2,The type of cell division process that occurs ...,0.555,0.673,0.608
3,The optimal environmental circumstances that f...,0.632,0.620,0.626
4,"During bacterial binary fission, the crucial s...",0.628,0.636,0.632
5,The fundamental distinction between a somatic ...,0.592,0.665,0.626
...,...,...,...,...
158,The human eye detects different colors through...,0.582,0.651,0.614
159,The significant benefits of possessing binocul...,0.647,0.597,0.621
160,The correct sequence from largest to smallest ...,0.534,0.713,0.611


#### Creating Pandas Dataframe to store calculations in CSV

In [237]:
goldenset = pd.concat([goldenset, generated_col], axis=1)

In [238]:
goldenset

,QUESTION,ANSWER,Generated Answer,Precision,Recall,F1
ID,,,,,,
1,Define 'asexual reproduction',Asexual reproduction is a type of reproductive...,Asexual reproduction is a process where an org...,0.652,0.765,0.704
2,Identify the sort of cell division that is inv...,The cell division in asexual reproduction is m...,The type of cell division process that occurs ...,0.555,0.673,0.608
3,Outline the ideal environmental conditions for...,Stable and uniform environments with a good su...,The optimal environmental circumstances that f...,0.632,0.620,0.626
4,List the key events in binary fission for bact...,The key events that happen in binary fission f...,"During bacterial binary fission, the crucial s...",0.628,0.636,0.632
5,What is the difference between a somatic cell ...,Somatic cells are all the diploid cells in the...,The fundamental distinction between a somatic ...,0.592,0.665,0.626
...,...,...,...,...,...,...
158,Explain how the eye detects different colours.,"There are three types of cone cells, each with...",The human eye detects different colors through...,0.582,0.651,0.614
159,Discuss the value of having binocular vision.,Binocular vision requires two eyes quite close...,The significant benefits of possessing binocul...,0.647,0.597,0.621
160,Arrange the following structures of the excret...,"The largest is kidney, then nephron, glomerulu...",The correct sequence from largest to smallest ...,0.534,0.713,0.611


In [239]:
#exporting the dataframe to a csv file
goldenset.to_csv(r'C:\Users\KD\Desktop\Git\hsc-llm\berteval-model2.csv')

### Generating AI responses (with AI QUERY REWRITING AND KEYWORD EXTRACTION) MODEL 3

In [ ]:
generated_answers = [] 
for question in questions:
    reworded_query = rewrite_query(question)
    kws = get_keywords(reworded_query)
    test_response = get_response(reworded_query, perform_retrieval(vector_store, kws[0]))
    generated_answers.append(test_response)

**What is meant by asexual reproduction, where an organism produces offspring that are genetically identical to itself without the involvement of another individual or genetic material from another source?**
('organism produces offspring', 0.0042542192213185686)
('asexual reproduction', 0.015380821171891606)
('meant by asexual', 0.02570861714399338)
('organism produces', 0.02570861714399338)
('produces offspring', 0.02570861714399338)
('genetically identical', 0.02570861714399338)
('individual or genetic', 0.02570861714399338)
('genetic material', 0.02570861714399338)
('reproduction', 0.09568045026443411)
('source', 0.09568045026443411)
('meant', 0.15831692877998726)
('asexual', 0.15831692877998726)
('organism', 0.15831692877998726)
('produces', 0.15831692877998726)
('offspring', 0.15831692877998726)
('genetically', 0.15831692877998726)
('identical', 0.15831692877998726)
('involvement', 0.15831692877998726)
('individual', 0.15831692877998726)
('genetic', 0.15831692877998726)
**What typ

In [ ]:
len(generated_answers)

162

#### Computing BERTSCORE FOR MODEL 3

In [ ]:
scorer = BERTScorer(model_type='bert-base-uncased')

In [ ]:
precision = []
recall = []
f1 = []
for x in range(len(generated_answers)):
    P, R, F1 = scorer.score([generated_answers[x]], [reference_answers[x]])
    precision.append(P.item())
    recall.append(R.item())   
    f1.append(F1.item())

In [ ]:
precision = [round(p, 3) for p in precision]
recall = [round(r, 3) for r in recall]
f1 = [round(f, 3) for f in f1]

In [ ]:
for i in range(len(precision)):
    print(f"Question {i+1} - Precision: {precision[i]} Recall: {recall[i]} F1: {f1[i]}")

Question 1 - Precision: 0.645 Recall: 0.804 F1: 0.716
Question 2 - Precision: 0.561 Recall: 0.626 F1: 0.592
Question 3 - Precision: 0.65 Recall: 0.647 F1: 0.649
Question 4 - Precision: 0.598 Recall: 0.671 F1: 0.633
Question 5 - Precision: 0.57 Recall: 0.656 F1: 0.61
Question 6 - Precision: 0.608 Recall: 0.608 F1: 0.608
Question 7 - Precision: 0.658 Recall: 0.573 F1: 0.613
Question 8 - Precision: 0.644 Recall: 0.636 F1: 0.64
Question 9 - Precision: 0.637 Recall: 0.63 F1: 0.633
Question 10 - Precision: 0.678 Recall: 0.658 F1: 0.668
Question 11 - Precision: 0.546 Recall: 0.597 F1: 0.57
Question 12 - Precision: 0.609 Recall: 0.563 F1: 0.585
Question 13 - Precision: 0.636 Recall: 0.649 F1: 0.642
Question 14 - Precision: 0.521 Recall: 0.492 F1: 0.506
Question 15 - Precision: 0.528 Recall: 0.638 F1: 0.578
Question 16 - Precision: 0.506 Recall: 0.53 F1: 0.517
Question 17 - Precision: 0.477 Recall: 0.633 F1: 0.544
Question 18 - Precision: 0.562 Recall: 0.579 F1: 0.571
Question 19 - Precision: 0

In [ ]:
index_list = []
for i in range(1, len(goldenset)+1):
    index_list.append(i)

In [ ]:
len(index_list)

162

In [ ]:
len(generated_answers)

162

In [ ]:
goldenset = pd.read_csv(r"C:\Users\KD\Desktop\Git\hsc-llm\ground_truth.csv", index_col=0)

In [ ]:
generated_col = pd.DataFrame({'ID':index_list,'Generated Answer': generated_answers, 'Precision': precision, 'Recall': recall, 'F1': f1})

In [ ]:
generated_col.set_index('ID', inplace=True)

In [ ]:
generated_col

,Generated Answer,Precision,Recall,F1
ID,,,,
1,"Asexual reproduction, also known as agamogenes...",0.645,0.804,0.716
2,The type of cell division process that occurs ...,0.561,0.626,0.592
3,To create an optimal environment for efficient...,0.650,0.647,0.649
4,The critical stages or milestones that occur d...,0.598,0.671,0.633
5,The primary distinction between a somatic cell...,0.570,0.656,0.610
...,...,...,...,...
158,The human eye detects and differentiates betwe...,0.642,0.634,0.638
159,The specific advantages of possessing binocula...,0.604,0.581,0.592
160,"The excretory system, also known as the urinar...",0.390,0.619,0.479


#### Creating Pandas Dataframe to store calculations in CSV

In [ ]:
goldenset = pd.concat([goldenset, generated_col], axis=1)

In [ ]:
goldenset

,QUESTION,ANSWER,Generated Answer,Precision,Recall,F1
ID,,,,,,
1,Define 'asexual reproduction',Asexual reproduction is a type of reproductive...,"Asexual reproduction, also known as agamogenes...",0.645,0.804,0.716
2,Identify the sort of cell division that is inv...,The cell division in asexual reproduction is m...,The type of cell division process that occurs ...,0.561,0.626,0.592
3,Outline the ideal environmental conditions for...,Stable and uniform environments with a good su...,To create an optimal environment for efficient...,0.650,0.647,0.649
4,List the key events in binary fission for bact...,The key events that happen in binary fission f...,The critical stages or milestones that occur d...,0.598,0.671,0.633
5,What is the difference between a somatic cell ...,Somatic cells are all the diploid cells in the...,The primary distinction between a somatic cell...,0.570,0.656,0.610
...,...,...,...,...,...,...
158,Explain how the eye detects different colours.,"There are three types of cone cells, each with...",The human eye detects and differentiates betwe...,0.642,0.634,0.638
159,Discuss the value of having binocular vision.,Binocular vision requires two eyes quite close...,The specific advantages of possessing binocula...,0.604,0.581,0.592
160,Arrange the following structures of the excret...,"The largest is kidney, then nephron, glomerulu...","The excretory system, also known as the urinar...",0.390,0.619,0.479


In [ ]:
#exporting the dataframe to a csv file
goldenset.to_csv(r'C:\Users\KD\Desktop\Git\hsc-llm\berteval-model3.csv')